<a href="https://colab.research.google.com/github/shstreuber/Data-Mining/blob/master/Module3_Regression2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Module 3: Regression**
In our last module, we saw that we can numerically relate two variables--either by evaluating how and when they vary together (covariance) or how variation in one variable leads to variation in another (correlation). It's now time for us to start thinking about how to determine more than just the mere **existence** of a relationship--we want to determine exactly **how** two variables relate to one another.

In this module, you will learn
* How to set up a simple linear regression
* How to set up a multiple regression
* How to set up a logistic regression
* How to interpret the output from a regression

**Be sure to expand all the hidden cells, run all the code, and do all the exercises--you will need the techniques for the lesson lab!**

#**What is Regression?**
Regression is one of the most valuable strategies in a data analyst's toolbox. It's widely used in the industry because it's quick to compute (unlike later models in this course) and therefore efficient on computing resources.

And it's not as hard as it may seem.

Did you know that there are at least **SEVEN most common types of regression**? Yes, seven.

<img src="https://theaxisofego.com/wp-content/uploads/2012/02/sevendwarfs.jpg?w=288&h=275">

As you read [the blog post under this link](https://www.analyticsvidhya.com/blog/2015/08/comprehensive-guide-regression/), focus on the **linear** and the **logistic** kind--that's what we will use this week.



#**0. Preparation and Setup**
We are working with our adult dataset again, so we're loading our libraries and our dataset just like last time, only that we won't need chisquare. This time around, we will use the following to allow us to use more statistics:
* **scipy.stats** -- all sorts of [statistical functions](https://docs.scipy.org/doc/scipy/reference/stats.html) that could potentially be useful
* **statsmodels** -- this is your typical [Python statistics package](https://www.statsmodels.org/stable/index.html). It is used for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data exploration.
* **ols** -- this is the Ordinary Least Squares function within statsmodels. It is located in the [formula class](https://www.statsmodels.org/stable/api.html#statsmodels-formula-api) and will help us set up our regression models.
* **Scikit Learn** -- we already met [this package](https://scikit-learn.org/) in this course. It is one of the most popular machine learning packages.

**The Dataset**

For this purpose, we will use the insurance prediction dataset. This dataset exists in TWO forms; the first one is the original, with [numeric and categorical attributes mixed](https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/insurance_with_categories.csv). This dataset contains 1338 observations (rows) and 7 features (columns). The dataset contains 4 numerical features (age, bmi, children and charges) and 3 categorical features (sex, smoker and region). Later, we will encounter another version of this dataset where [the categories are converted](https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/insurance.csv) into factors with numerical value designated for each level. Lastly, the **dependent attribute**, i.e. the **class attribute** in this datasetis charges, which tells us how high a filed claim was.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import spatial
import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from IPython.display import IFrame  # This is just for me so I can embed videos

#Reading in the data as adult dataframe
insurance = pd.read_csv("https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/insurance_with_categories.csv")

#Verifying that we can see the data
insurance.head()

In [ ]:
# And let's verify the datatypes before we go off into analysis land:
insurance.dtypes

Let's get started!

#**1. Exploratory Data Analysis**
 Let's get a sense of how some of the independent numeric variables relate to our class attribute (i.e. the one attribute most likely to change when the independent attributes change)--in this case, our **class attribute** is charges.

In [ ]:
plt.scatter(insurance['age'], insurance['charges'], alpha=0.5) # alpha=0.5 is added to make the points slightly transparent, making it easier to see overlapping points.
plt.title('Scatter plot age vs. charges')
plt.xlabel('age')
plt.ylabel('charges')
plt.show()

Hm. Interesting. We see three somewhat defined bands here. The solid band at the bottom, followed by a less solid band starting between 10,000 and 20,000, and then, there's the band starting between 30 and 40k. So, the older a person gets, the more likely they are to incur higher insurance claims. Given what we know about getting older, that seems about right.

But what about the dependency of [bmi](https://www.nhlbi.nih.gov/health/educational/lose_wt/BMI/bmicalc.htm) and charges? Might it be reasonable to assume that higher bmi = higher charges?  Well, let's see ...

In [ ]:
plt.scatter(insurance['bmi'], insurance['charges'], alpha=0.5)
plt.title('Scatter plot bmi vs. charges')
plt.xlabel('bmi')
plt.ylabel('charges')
plt.show()

**SURPRISE!** Higher bmi does not always mean higher insurance charges! We see plenty of data in the 40+ ranges that seem to have identical charges to data in the 20 to 25 range. HOWEVER, we also see more data breaking the 30,000 barrier starting at a bmi of 30+. So ... two relationships, really?

Next, let's check the correlations between all the attributes:

In [ ]:
insurance.corr(numeric_only=True) # Need to specify numeric_only=True because the default has changed with version 2.0

## Your Turn
Look at the table above. Which attribute has the best correlation to charges? **Display just this attribute** in this field:

##Your Turn
Now, does having more children mean higher charges? Let's see!

Use the field below to **build a scatterplot of children and charges**! What does the relationship reveal?

Really think about this--wouldn't it be interesting if we could find the rule that connects these variables?

# **2. Simple Regression**
In the previous module, we saw how Correlation and the Pearson Correlation Coefficient, r, give us a first insight into the relationship between two variables. But that is only 1 number which doesn't tell us anything about the relationship of each single datapoint with the next--and it certainly doesn't allow us to predict any future values.

That is why Regression is a better aproach.

To review the **concepts and vocabulary** of simple regression, watch this video:

In [163]:
IFrame(src="https://www.youtube.com/embed/aq8VU5KLmkY", width=560, height=315)


##**2.1 Simple Linear Regression with statsmodels**
The simplest regression comes in the form of

                    `y = f(x) + b`

where x is the independent variable and y the dependent variable (aka the class attribute). The idea is that you can find a coefficient for x (here named f) that, when paired with a constant value (here b), will allow you to calculate any dependent value y. Since there is no exponential math going on here, the result will be a straight line. That's also why we call this "linear regression."

So, how does simple linear regression work in Python? Well, you've already taken the first steps by plotting the independent variable x on the dependent variable y above. The rest is explained in the instructor video below:

In [164]:
IFrame(src="https://www.youtube.com/embed/jC1z8pIAsp8", width=560, height=315)

###**2.1.1 Simple Linear Regression with statsmodels**
Below, we will use Ordinary Least Squares to calculate our regression equation. As the video above explains, Ordinary Least Squares, or linear least squares, estimates the parameters in a regression model by minimizing the sum of the squared residuals. This method draws a line through the data points that minimizes the sum of the squared differences between the observed values and the corresponding fitted values.

**If you can't remember what residuals are**, check out [this blog post](https://www.statisticshowto.com/residual/).

So, let's go back to age and insurance charges! Note that, in order to format our output as

                          `y = f(x) + b`

We need to make sure that the calculation considers the constant b. That's why we use add_constant below. Otherwise, all we would get as output is the coefficient f.

In [ ]:
X = insurance['age']
y = insurance['charges']
X = sm.add_constant(X)
insurance.mod1 = sm.OLS(y, X).fit()  ## sm.OLS(output, i.e. dependent variable, input, i.e. independent variable)
insurance.mod1_summary = insurance.mod1.summary()
print(insurance.mod1_summary)

### **WHOA! T.M.I.!--Understanding the Output**

We're seeing a lot of information here.

**Most important**: Point your eyes a little below the middle of this output. That's where you'll see the following:
```
         coef	    std err	 t	    P>|t|	[0.025	  0.975]
const	3165.8850	937.149	3.378	0.001	1327.440	5004.330
age	   257.7226	22.502	11.453	0.000	 213.579	301.866
```
**What does this mean?**

The first number here is const coef, that is, the constant in our linear equation, here 3165.8850. It is the value for our letter b in

                               `y = f(x) + b`

The second important number here is age coef, that is, the factor by which we multiply the independent variable, here 257.7226. It is the value for f. So, the equation that OLS has built for us is

                              `y = (257.7226 * x) + 3165.8850`


The coefficient of 257.7226 means that as the age variable increases by 1, the predicted value of charges increases by 257.7226 plus 3165.8850.  

We also meet an "old friend" again--the p value in the P>|t| column. Since the initial assumption, that is, the H0, of a Regression is always that all variables are independent from each other, the p value shows us how many %  of the result are really due to random chance. Remember how we learned that the magic threshold is 5%, and "if p is low, H0 must go"? With p being at 0.000, we're seeing here that age and charges are obviously dependent.

**Now, let's look at this:**
```
                         OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.089
....
No. Observations:                1338   AIC:                         2.883e+04
Df Residuals:                    1336   BIC:                         2.884e+04                                         
==============================================================================

```
Let's start at the bottom: The number of observations is 1338, which, since we are looking at 2 attributes, makes the degrees of freedom 1338 - 2 = 1336. No problem here. Let's move up. The R-squared value of 0.089 or 8.9% means that 8.9% of the variation in charges can be explained by using age as the predictor.

To learn more about how to interpret the entire output, check out [this excellent video](https://www.youtube.com/watch?v=VvlqA-iO2HA).

**Wait, what?**

Let's look at these numbers again:

In [ ]:
print('Parameters: ', insurance.mod1.params)
print('R2: ', insurance.mod1.rsquared)

The R-squared value is 8.9%! If we compare this to our initial impression from the scatter diagram and the p-value interpretation above, namely that age could well be closely related to charges, th R-squared value makes age a lousy predictor of most of the charges. **LOUSY**!

Let's take a look at the calculated values for x (the video called them y "hat") and then plot them:

In [ ]:
predictions = insurance.mod1.predict(X) # Here, you'll see the CALCULATED values with the formula
predictions

###**2.1.2 Plotting the results**
Now let's build all four regression plots. We pass in the model as the first parameter, then specify the predictor variable we want to analyze.

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(insurance.mod1, "age", fig=fig)

**WOW**! Remember the R-squared number of 8.9%? Here you can see the distribution of values for the age variable. Look at the plot in the upper left-hand corner of the fitted, i.e. predicted vs. actual values. That fit line isn't all that representative, is it--although there's clearly some connection (so, the p-value was correct)? And the residuals, which should be centered around 0? Check out the upper right-hand corner graph--obviously not the case, is it?

## **2.2. Multiple Linear Regression with scikit learn**--Numeric Attributes

Well. We've seen where looking at just ONE variable in connection with our insurance charges got us--not too far. So ... what if we look at a few variables together as joint influencers of the charge variable? Perhaps that will increase the R-squared value. When we run a regression with more than one independent attribute in the X variable, that is called Multiple Regression. Here, we are using the linear model, so we are performing Multiple Linear Regression.

Now, we could do this in statsmodels--but that'll be your job later. To shake things up, we will use another very popular and versatile package for this purpose: Scikit Learn. In our preparation and setup section, we already imported its linear_model subpackage.

We will focus our Multiple Regression on the **joint influence of age, bmi, and number of children on charges**. You have already encountered these three in section 1 of this workbook.

In [ ]:
# Let's use the three "big" predictor variables from above: age, bmi, and number of children

X = insurance[['age','bmi','children']] # Now we have values from three different independent attributes stored in X
y = insurance['charges'] # This is, again, our dependent attribute
print(X) # This is the content of our X variable

In [170]:
# Defining the model for a multiple regression using the lm.fit() function:

lm = linear_model.LinearRegression() # Here, we are using the linear_model subpackage from scikit-learn that we imported above
model = lm.fit(X,y) # And here, we are building the model

You will have noticed that, when you clicked on the previous code cell, the output was ... nothing. When we run a linear regression with SKLearn, we don’t get a pretty table like in Statsmodels. But then, the model is much easier to build. What we can do is use built-in functions to return the score, the coefficients and the estimated intercepts. Let’s see how it works:

In [ ]:
# To find the R-squared score:
lm.score(X,y)

In [ ]:
# To find the coefficients for the predictors:
lm.coef_

In [ ]:
# To find the intercept:
lm.intercept_

What does all of this mean? Well, the R-squared value has improved from 8.9% to 12%. That's something. We are seeing our coefficients and the intercept, as well. This makes our equation


```
charges = 239.99 * age + 332.08 * bmi + 542.86 * children - 6916.24

```
As I said: Better already. Good? Not good enough. There's got to be a better way. And there is.


## **2.3 Multiple Linear Regression--All Attributes**
Remember the other attributes we have in this dataset? Let's take a quick look:

In [ ]:
insurance.info()

We still have sex, smoker, and region! Let's see what their influence is when combined with our numeric attributes.

First, we set up our X and y attributes again.

In [175]:
X = insurance.iloc[:, :-1].values
y = insurance.iloc[:, 6].values
print(X)
print(y)

[[19 'female' 27.9 0 'yes' 'southwest']
 [18 'male' 33.77 1 'no' 'southeast']
 [28 'male' 33.0 3 'no' 'southeast']
 ...
 [18 'female' 36.85 0 'no' 'southeast']
 [21 'female' 25.8 0 'no' 'southwest']
 [61 'female' 29.07 0 'yes' 'northwest']]
[16884.924   1725.5523  4449.462  ...  1629.8335  2007.945  29141.3603]


Now, we can run our Multiple Regression again--this time with a process called "Backward Elimination." In short, **Backward Elimination** is the process of entering all the independent variables into the equation first and **delete them one at a time** if they do not contribute to the regression equation.

**Time to get up close and personal with r-squared**! Watch the video below:



In [203]:
IFrame(src="https://www.youtube.com/embed/bMccdk8EdGo", width=560, height=315)

**OH!**
But since regression requires only numeric variables, we will need to work with an entirely numeric dataset. Good thing we have one--otherwise, we would need to do some serious preprocessing. Let's load this dataset:

In [ ]:
insurance2 = pd.read_csv("https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/insurance.csv")
insurance2.head()

There is an additional column called "insuranceclaim". We will delete this column for now. (NOTE: If you click the "run" icon below more than once, you will get a runtime error because Python is trying to delete a column that has already been deleted).

In [ ]:
del insurance2['insuranceclaim']
insurance2.head()

Now we can start eliminating attributes if they don't contribute to an optimal regression result (again, let's watch r-squared). We will use statsmodels this time around. First, we will set up X and y again so that we replace the strings currently in these variables with numbers:



In [ ]:
X = insurance2.iloc[:, :-1].values
y = insurance2.iloc[:, 6].values

X = np.append(arr = np.ones((1338, 1)).astype(int), values = X, axis = 1) # We are building our numpy array

X_opt = X[:, [0, 1, 2, 3, 4, 5, 6]] # Now we combine all 6 input variables into our first iteration
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit() # Now we set up our regressor function with OLS again as before.
print(regressor_OLS.summary())

Wow! With all variables combined, we can achieve and r-squared value of 75%. Nice. But you never know if two variables maybe negatively impact each other. So, let's remove one and see if that will help us increase r-squared.

In [ ]:
# How about age? That wasn't so good before.

X_opt = X[:, [0, 2, 3, 4, 5]] # Age has been removed
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit() # Now we set up our regressor function with OLS again as before.
print(regressor_OLS.summary())

Oh. Well, that actually decreased R-squared. How about eliminating sex?

In [ ]:
# bmi looked pretty split before. Removing bmi now.

X_opt = X[:, [0, 2, 4, 5, 6]] # Age has been removed
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit() # Now we set up our regressor function with OLS again as before.
print(regressor_OLS.summary())

Wow. That dropped the r-squared function even faster. How about bmi, then?

In [ ]:
X_opt = X[:, [0, 1, 3, 4, 5, 6]] # Removed bmi
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit() # Now we set up our regressor function with OLS again as before.
print(regressor_OLS.summary())

Heh. Removing bmi did nothing to our r-squared--except for saving us the computing resources that would come from applying the regression to the full dataset including bmi.

In [ ]:
X_opt = X[:, [0, 1, 3, 4, 6]] # Removing bmi and region
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit() # Now we set up our regressor function with OLS again as before.
print(regressor_OLS.summary())

Well, looks like we definitely want to keep region! What about smoker?

In [ ]:
X_opt = X[:, [0, 1, 3, 5, 6]] # Removing bmi and smoker
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit() # Now we set up our regressor function with OLS again as before.
print(regressor_OLS.summary())

Wow. We just saved ourselves processing 2,600 values and lost less than 1 percent in our r-squared value. That seems like a pretty good deal!

##**WAIT! What was the Goal again?**

We started this entire regression journey because we wanted to find a function that would allow us to **PREDICT** future values of the charges attribute. This means that we would want to combine all our X inputs (as we did above) in order to find y values based on the regression formula. This also means that we need more independent attributes with no values in the associated y attribute, so that we can actually **CALCULATE** y.

But our dataset is clean, with 1338 values in each column.

So ... we need to either invent more independent attribute values, OR we need to delete some of those y attribute data, so that we can actually calculate them.

Inventing data is a big no-no when you're dealing with actual data (although data scientists will often create artificial datasets to prove a theory). This means that the only other thing left to us is to delete data from the y column.

But how? And how much?



##**2.4 Training and Test Datasets**
What we really need at this point is two different datasets. We need one dataset to build, or **TRAIN**, our regression function. Then, we can use a second dataset on which to **TEST** whether our function actually works. However, since we have only one dataset, we will have to
* Split the dataset into a larger training dataset and a smaller testing dataset
* Build our function on the basis of the training dataset
* Remove the y values in the test dataset
* Apply our function to the test dataset
* Compare the computed values for the test dataset (the Y hat values!) with the actual values in y that we did, of course, not throw away entirely.

In fact, that's what we will do with all data from hereon out.

###**2.4.1 Building the Training and Test Set**
We will use the generic train_test_split function from scikit-learn for this. We will build a separate X_train and X_test frame and a y_train and a y_test frame.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_opt, y, test_size = 0.2, random_state = 0) # Size of test data is 20%, size of training data is 80%

print('X Train', X_train.shape)
print('X Test', X_test.shape)

###**2.4.2 Finding the Regression Formula**
Now we can find the regression formula in the training set!

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

print('Model accuracy score:', round(regressor.score(X_train,y_train)*100,2))

The model accuracy score is basically the r-squared value expressed as percentage and rounded to two decimals. Good.

###**2.4.3 Applying the Regression Formula to the Test Set**
Now we can plug our X_test set values into the formula and calculate the y_test values to predict. In order not to confuse the actual y_test values with the values we are going to calculate, we will call our predicted values y_pred.

In [ ]:
y_pred = regressor.predict(X_test)
print(y_pred[0:10])

###**2.4.4 Comparing predictions and actual values in the test set**
What good are the predictions if you can't tell how close to reality they are? That's where our y_test comes in. Now we can compare our y_pred and y_test, and we will see how good our prediction is.

In [ ]:
charges_pred = pd.DataFrame({'Prediction (y_pred)':y_pred,'Actual Values (y_test)':y_test}).round(2)
print(charges_pred.shape)
charges_pred.head(25)

We can see that the y_pred data are a bit more extreme on each end than the y_test data. That's why accuracy is at around 73%--not great, but certainly better than 8.9% or 12%.

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*Do0lysWTXigG3F79FRRtKQ.png">

# **3. Classification with Logistic Regression**
You just learned how to use Simple Regression and a training/ test set split to calculate and predict **numeric output**. That's because Regression is a mathematical tool.

But what if we are interested in **categorical output**? What if we want to use a student's grades to predict whether they will pass or fail a class at the end of the semester? Or use a soccer team's past number of goals in a tournament to determine if it will win or lose the World Cup? Or see if  body weight, calorie intake, fat intake, and age have an influence on the probability of having a heart attack (yes vs. no)?

We will use the first insurance dataset again, but with an additional insuranceclaim dimension that contains 1 (claim valid) or 0 (claim invalid). We are setting the dataframe up from scratch again first, though.

##**3.1 What is Logistic Regression?**
We have already seen the questions Logistic Regression tries to answer. Basically, there are THREE RULES for Logistic Regression:
1. The dependent variable should be dichotomous (e.g., yes/ no, present/ absent, pass/ fail, and so on).
2. There should be no outliers in the data.
3. There should be no high correlations (multicollinearity) among the predictors (i.e. we'll have to set up a correlation matrix among the predictors and look for small Pearson Correlation Coefficients (r, with which we have already worked multiple times).

###**How does Logistic Regression Work?**
Logistic Regression isn't all that different from Multiple Linear Regression. But where the output of Multiple Linear Regression is continuous, the output of Logistic Regression is binary or "dichotomous." In other words, we don't want a number at the end; we want one of two values. That requires some additional math--namely [a logistic function](https://careerfoundry.com/en/blog/data-analytics/what-is-logistic-regression/), which is based on calculating the odds of an outcome. For example, let's say that, since in the past your soccer team has won 20 out of 21 games in the World Cup tournament, the odds of them winning finals are 20 to 21, or, when expressed as an odds ratio, they are 20/21. To standardize these odds ratios to a 1 or 0 outcome, we need the *logit function*, which is there Logistic Regression gets its name. This function applies a log odds logarithm to the outcome of a multiple regression.

To learn more about the math behind Logistic Regression, watch this great 9-minute video:


In [189]:
IFrame(src="https://www.youtube.com/embed/yIYKR4sgzI8", width=560, height=315)

##**3.2 Working with Logistic Regression**
Here is how Logistic Regression works for our example:

Remember how, when working with the insurance2 dataset, we deleted that insuranceclaim column? Well, we are going to use it now. For that purpose, we are reading the dataset in again and name it insurance3.

In [ ]:
insurance3 = pd.read_csv("https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/insurance.csv")
insurance3.head()

###**3.2.1 Satisfying the Three Rules**
We already see that insuranceclaim is binary or dichotomous and seems to have values of 1 and 0. That satisfies Rule #1 above. On to our second rule--we need to check for multicollinearity, i.e. connections among the attributes themselves. For that, we are setting up a correlation matrix:

In [ ]:
corr = insurance3.corr()
corr

##Your Turn
Which variables have > 50% correlation? Type the variable below:

To make things fancy, we can even visualize the correlations in a heat map using the [seaborn package](https://seaborn.pydata.org/introduction.html), which is one of the really great graphics packages in Python! This works like so (and will give you the answer to the Your Turn question above):

In [ ]:
import seaborn as sns

f, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(corr, mask = np.zeros_like(corr, dtype = bool),
            cmap = sns.diverging_palette(150, 275, as_cmap = True), square = True, ax = ax)

Compare what you see on this heatmap of correlations with the answer you typed in the field above. Does your answer match what we see here?

###**3.2.2 Preprocessing the Data: Training and Test Set**
The data set consists of record of 1338 patients in total. To train our model we will be using 1000 records. We will be using 300 records for testing, and we will use the last 38 records to cross check our model. Instead of splitting percentage-wise, as we did above, we are now splitting by row number:

In [193]:
dfTrain = insurance3[:1000]
dfTest = insurance3[1000:1300]
dfCheck = insurance3[1300:]

Next, we separate the label and features (for both training and test dataset). In addition to that, we will also convert them into NumPy arrays as our machine learning algorithm processes data in NumPy array format.

In [ ]:
# Convert 'insuranceclaim' column to numpy array for training and testing sets
train_y = np.asarray(dfTrain['insuranceclaim'])
train_x = np.asarray(dfTrain.drop('insuranceclaim', axis=1)) # We remove insurancelcaim from the predictors (because it's the output variable)

test_y = np.asarray(dfTest['insuranceclaim'])
test_x = np.asarray(dfTest.drop('insuranceclaim', axis=1))

# Display train_x to verify the changes
train_x

This is what the numpy array looks like. No worries. All the data is there.

As the final step before using machine learning, we will normalize our inputs. Machine Learning models often benefit substantially from input normalization. It also makes it easier for us to understand the importance of each feature later, when we’ll be looking at the model weights.  We’ll standardize the data such that each variable has 0 mean and standard deviation of 1.

One way of doing this is by using the means:

In [195]:
means = np.mean(train_x, axis=0)
std = np.std(train_x, axis=0)

train_x = (train_x - means)/std
test_x = (test_x - means)/std

Another way of doing this is with a function called StandardScaler that we need to import (because we didn't import it at the beginning of this file):

In [196]:
from sklearn.preprocessing import StandardScaler

# Standardize the numeric columns
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

Now, we can start with our Logistic Regression!

###**3.2.3 Building the Model**
We can now train our classification model with logistic regression. First, we create a model and then use the fit function to train the model. We use the [LogisticRegression() function](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) from scikit-learn. Follow the link to learn more about this very versatile function!

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_x, train_y)

###**3.2.4 Testing the Model**
Now we use our test data to determine the accuracy of the model.

In [ ]:
accuracy = model.score(test_x, test_y)
print("accuracy = ", accuracy * 100, "%")

###**3.2.5 Using the Model for Predictions**
Now we can use the model for predictions on the test set!

In [ ]:
pred = model.predict(test_x)
print(pred)

And here is our entire analysis process--and the calculated output! We are seeing how the function allowed us to predict 1 or 0, just as we needed!

In [ ]:
# Let's compare the actual y and the predicted y

realvsmodel = pd.DataFrame(pred,test_y)
realvsmodel = pd.DataFrame({'predicted':pred,'original':test_y})
realvsmodel.head(25)

##Your Turn
1. Create a subset of the adult dataset with just the numeric variables.
2. Can you use Logistic Regression to predict the sex of someone based on the all the numeric inputs? Use this as your guiding question for your exploration. Follow all the steps in section 3.2.

#**4. If you get stuck ...**

In [ ]:
# Scatterplot children vs charges
plt.figure(figsize=(10, 6))
plt.scatter(insurance['children'], insurance['charges'], alpha=0.5) # alpha=0.5 is added to make the points slightly transparent, making it easier to see overlapping points.
plt.title('Scatter Plot of Children vs. Charges')
plt.xlabel('Number of Children')
plt.ylabel('Charges')
plt.grid(True)
plt.show()

1. Exploratory Data Analysis

	        age	      bmi	      children	charges

age	      1.000000	0.109272	0.042469	**0.299008**

bmi	      0.109272	1.000000	0.012759	0.198341

children	0.042469	0.012759	1.000000	0.067998

charges	 **0.299008**	0.198341	0.067998	1.000000

3. 'charges' and 'smoker' have a correlation of 78%.

In [ ]:
# 3. Create a subset of the adult dataset with just the numeric variables.
# Can you use Logistic Regression to predict the sex of someone based on the all the numeric inputs? Use this as your guiding question for your exploration.
# Follow all the steps in section 3.2.

# Load the dataset
insurance3 = pd.read_csv("https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/insurance.csv")

# Create a subset of the dataset with numeric variables and 'sex'
insurance_num = insurance[['age', 'bmi', 'children', 'charges', 'sex']]

# Split the data into training and testing sets
dfTrain1 = insurance_num[:1000] # Reducing the dataframe to 1000 rows for the training set
dfTest1 = insurance_num[1000:1300] # Defining the test set
dfCheck1 = insurance3[1300:] # Everything else

# Extract the target variable ('sex') and the predictors
train_y = np.asarray(dfTrain1['sex'])
train_x = np.asarray(dfTrain1.drop('sex', axis=1))
test_y = np.asarray(dfTest1['sex'])
test_x = np.asarray(dfTest1.drop('sex', axis=1))

# Standardize the numeric columns
means = np.mean(train_x, axis=0)
std = np.std(train_x, axis=0)
train_x = (train_x - means) / std
test_x = (test_x - means) / std

# Train a logistic regression model
model = LogisticRegression()
model.fit(train_x, train_y)

# Evaluate the model
predictions = model.predict(test_x)
accuracy = accuracy_score(test_y, predictions)

print("Accuracy:", accuracy * 100, "%")
print("Predictions:", predictions)

# Create a DataFrame to compare predictions with actual values
real_vs_predicted = pd.DataFrame({'Predicted': predictions, 'Actual': test_y})
realvsmodel.head(25)
# print(real_vs_predicted.head(25)) # Another way to show the values on screen
